# A. Setup
In the following section, I'll do some initial housekeeping in order to make sure the kernel runs with the correct library versions and stays machine agnostic.

## A.1 Conda environment
I use Conda for my kernels since it is a standard toolchain for data scientists and provides a good all in one solution for all types of common requirements.

In [1]:
import os

try:
    if bool(os.environ["DEEPNOTE"]):
        import sys
        sys.path.append('/usr/local/lib/python3.9/site-packages/')
        !sudo conda install -y pandas
except KeyError:
    !conda env create -f ../environment.yml && conda activate ntnu-dpnv-assignment-1


CondaValueError: prefix already exists: /Users/aakash_mallik/miniconda3/envs/ntnu-dpnv-assignment-1



In [106]:
import pandas as pd
import sqlite3
from pathlib import Path

## A.2 Global variables

In [107]:
DATA_DIR = Path("../data/")

# B. Data
In the following section I'll be loading and peeking into the data provided in the assignment.

## B.1 Reading
I'll go ahead and read the following files to collect data:
- real_estate.csv
- colors.json
- books.xml
- chinook.sqlite3

**Note:** We are not reading the entire database in case of SQLite3 data. Usually databases hold a huge amount of data and loading the entire data into a dataframe (which resides on RAM) is not a good idea.

In [108]:
# Reading CSV
data_real_estate = pd.read_csv(DATA_DIR / "real_estate.csv")

# Reading JSON
data_colors = pd.read_json(DATA_DIR / "colors.json")

# Reading XML
data_books = pd.read_xml(DATA_DIR / "books.xml")

# Reading SQLite3 (First 5 entires only)
db_connection = sqlite3.connect(DATA_DIR / "chinook.sqlite3")
query = 'SELECT firstname, lastname, address FROM customers LIMIT 5;'
data_chinook = pd.read_sql_query(query, db_connection)

## B.2 Peeking
I'll peek into the data and display first 5 rows of each.

### a. Real Estate Data

In [109]:
data_real_estate.head(5)

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768


### b. Colors data

In [110]:
data_colors.head(5)

,color,value
0,red,#f00
1,green,#0f0
2,blue,#00f
3,cyan,#0ff
4,magenta,#f0f


### c. Books data

In [111]:
data_books.head(5)

,id,author,title,genre,price,publish_date,description
0,bk101,"Gambardella, Matthew",XML Developer's Guide,Computer,44.95,2000-10-01,An in-depth look at creating applications \n ...
1,bk102,"Ralls, Kim",Midnight Rain,Fantasy,5.95,2000-12-16,"A former architect battles corporate zombies, ..."
2,bk103,"Corets, Eva",Maeve Ascendant,Fantasy,5.95,2000-11-17,After the collapse of a nanotechnology \n ...
3,bk104,"Corets, Eva",Oberon's Legacy,Fantasy,5.95,2001-03-10,"In post-apocalypse England, the mysterious \n ..."
4,bk105,"Corets, Eva",The Sundered Grail,Fantasy,5.95,2001-09-10,"The two daughters of Maeve, half-sisters, \n ..."


### d. Chinook data
Since we only extracted 5 entries from the DB, we don't need `head` in this case and can display the entire dataframe.

In [112]:
data_chinook

,FirstName,LastName,Address
0,Luís,Gonçalves,"Av. Brigadeiro Faria Lima, 2170"
1,Leonie,Köhler,Theodor-Heuss-Straße 34
2,François,Tremblay,1498 rue Bélanger
3,Bjørn,Hansen,Ullevålsveien 14
4,František,Wichterlová,Klanova 9/506


# C. Data exploration
In the following section, we'll inspect the data further to gain some insights on what we are dealing with. As stated in the assignment, we'll focus only on the `house` data - `data_real_estate`.

## C.1 Number of rows

In [113]:
num_rows = data_real_estate.shape[0]
print(f"Number of rows in the CSV file: {num_rows}")

Number of rows in the CSV file: 985


## C.2 Prices

In [114]:
prices = data_real_estate['price']

We'll go ahead and run some assertions to make sure our steps are correct.

In [115]:
# Check if prices is indeed an instance of the Series class from pandas
assert type(prices) == pd.core.series.Series

# Check if we have exactly 985 items in the series
assert len(prices) == 985

# Verify if the 10th item in the series matches with the content of the CSV
assert prices[9] == 98937

## C.3 Basic statistics

In [116]:
prices_mean = prices.mean()
prices_sd = prices.std()

print(
    f'Mean of prices: {price_mean:.6f}\nStandard deviation of prices: {prices_sd:.6f}')

Mean of prices: 234144.263959
Standard deviation of prices: 138365.839085


In [117]:
prices.describe()

count       985.000000
mean     234144.263959
std      138365.839085
min        1551.000000
25%      145000.000000
50%      213750.000000
75%      300000.000000
max      884790.000000
Name: price, dtype: float64

We make sure our mean calculation is correct by writing a small asssertion. Since floating point number comapration is usually unpredictable and unreliable due to rounding errors, our assertion will check for the closest integer.

In [118]:
assert round(prices_mean) == 234144
assert round(prices_sd) == 138366

## C.4 Filter
We will find:
- max price in the entire data set
- find the most expensive house
- find how many square feet it has and store it in a variable

In [119]:
max_price = prices.max()
max_price_house = data_real_estate[data_real_estate['price'] == max_price]

# Since there can be more than one house with the max_price, 
# we'll assert and confirm that our filter only returned one
assert max_price_house.shape[0] == 1, "More than one value found"

most_expensive_area = max_price_houses.iloc[0]['sq__ft']

print(f"The most expensive area: {most_expensive_area} square feet")

The most expensive area: 4400 square feet


Checking for correctness

In [120]:
assert most_expensive_area == 4400

## C.5 Column Manipulation

In [121]:
data_real_estate['sqft_price'] = data_real_estate['price'] / data_real_estate['sq__ft']
cheapest_house = data_real_estate[
    data_real_estate['sqft_price'] == data_real_estate['sqft_price'].min()
]

Checking for correctness

In [122]:
assert type(cheapest_house) == pd.core.frame.DataFrame
assert len(cheapest_house) == 1
assert cheapest_house["price"].item() == 2000 
assert cheapest_house["sq__ft"].item() == 5822

## C.6 Unique Values 
We will now find out number of multi-family houses that had less than 3 bathroom.

In [123]:
data_real_estate["type"].unique()

array(['Residential', 'Condo', 'Multi-Family', 'Unkown'], dtype=object)

As we can see, one of the types is `Multi-Family`, so we will use it in our filter.

In [124]:
few_br_multif = data_real_estate[
    (data_real_estate["type"] == 'Multi-Family') &
    (data_real_estate["baths"] < 3)
]
num_few_br_multif = few_br_multif.shape[0]

## C.5 Column conversions
Now we'll convert an entire column and create a new one, finally adding it to our original dataframe.

**Note:** 1 foot = 0.3048 meters

In [125]:
data_real_estate['area_m2'] = data_real_estate['sq__ft'] * (0.3048 ** 2)

data_real_estate.head(3)

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude,sqft_price,area_m2
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879,70.839713,77.666941
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028,58.450728,108.417848
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839,86.532663,73.950820


## C.7 Grouping
For our final task, we will group cities and sum up the areas of the houses in each of them, giving us the total area sold per city. We'll finally sort them.

**Note:** The squeeze at the end is important since without it, we would endup with a Dataframe, but the question asks for a series.

In [178]:
sorted_grouped_city_area = data_real_estate.groupby('city').aggregate(
    total_area_sold=('area_m2', 'sum')
).sort_values(
    by=['total_area_sold'], 
    ascending=False
).squeeze()

sorted_grouped_city_area

city
SACRAMENTO         56864.092723
ELK GROVE          19579.780195
ANTELOPE            5262.864313
ROSEVILLE           4587.273406
CITRUS HEIGHTS      4255.330844
RANCHO CORDOVA      3543.136140
FOLSOM              3077.227394
GALT                3045.175845
CARMICHAEL          2885.754228
NORTH HIGHLANDS     2244.258737
ORANGEVALE          1683.495988
FAIR OAKS           1387.785612
ROCKLIN             1387.692708
RIO LINDA           1354.061808
WILTON              1304.265779
LINCOLN              895.492403
GRANITE BAY          828.880923
PLACERVILLE          792.091319
GOLD RIVER           607.400076
RANCHO MURIETA       550.543415
SLOUGHHOUSE          540.881499
EL DORADO HILLS      498.796422
ELVERTA              461.356497
AUBURN               417.413359
POLLOCK PINES        370.125711
LOOMIS               274.249774
GREENWOOD            264.402052
EL DORADO            247.493699
MATHER               194.446063
CAMERON PARK         187.292529
WALNUT GROVE         160.443550
COO

top_five_cities = sorted_grouped_city_area[0:5].squeeze()
top_five_cities

Checking for correctness

In [191]:
assert type(sorted_grouped_city_area) == pd.core.series.Series
assert type(top_five_cities) == pd.core.series.Series
assert round(top_five_cities['ANTELOPE']) == 5263
assert sorted_grouped_city_area.nlargest(3).index[2] == 'ANTELOPE'

***

# D. Reflection

#### Q.1 How did the assignment go? Was it easy or hard?
**A.** It was great - A good warmup. It was easy for me since I have some experience with data wrangling.


#### Q.2 How many hours did you spend on it?
**A.** One and a half hours. Half an our more just to recheck my code.


#### Q.3 What was the most time-consuming part?
**A.** The last question took me a little bit more time than the others since it was the toughest to solve.


#### Q.4 If you need to do similar things later in your professional life, how can you improve? How can you do it more efficiently?
**A.** Since a programmer is not supposed to remember all the APIs and functions he uses, it is a good idea to note hints and function calls somewhere so that it is easier to find them next time if a similar problem comes up. Also in my experience, it is really very important to keep ones code repositories accesible. My old code serves as a great reference and I can easily recreate and redo things quickly if I have already done them before.


#### Q.5 Was there something you would expect to learn that this exercise did not include?
**A.** No, I think everything was as expected and I am looking forward to learn more from the upcoming exercises.


#### Q.6 Was there something that does not make sense?
**A.** Some of the entries don't have realistic data, for example some of the houses have 0 bedrooms, and 0 sq feet area. This was slightly unexpected but I think this is how raw data will be presented to us in real life too, so its a good learning experience.